In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote

In [2]:
import unicodedata
import re
from tqdm.notebook import tqdm

In [3]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver')
driver.get(insta_url)
time.sleep(1)

In [4]:
with open('password.txt') as f:
    password = f.read()

In [5]:
email = 'ckiekim'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [7]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [8]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

In [9]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [10]:
driver.get(url)
time.sleep(5)

In [11]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [12]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# 1) 본문 내용 가져오기

try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'.🍺삼도동 집아페🍺나만 알고 싶은 아담한 로컬 이자카야 찐맛집둘이 가도 좋고 혼술 감성에도 딱이고👍👍안주가 너무 맛있어서 술이 술술 들어가버려~@함께 갈 친구 소환@주소 : 제주 제주시 서광로 13길 3..#제주맛집 #제주도맛집 #제주공항근처맛집 #제주시맛집 #애월맛집 #중문맛집 #제주공항맛집 #서귀포맛집 #산방산맛집 #신제주맛집 #제주시청맛집 #제주애월맛집 #월정리맛집 #제주여행 #제주핫플'

In [14]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#제주맛집',
 '#제주도맛집',
 '#제주공항근처맛집',
 '#제주시맛집',
 '#애월맛집',
 '#중문맛집',
 '#제주공항맛집',
 '#서귀포맛집',
 '#산방산맛집',
 '#신제주맛집',
 '#제주시청맛집',
 '#제주애월맛집',
 '#월정리맛집',
 '#제주여행',
 '#제주핫플']

In [15]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-12-06'

In [16]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

'1,143'

In [17]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [18]:
row = [content, date, like, place, tags]

In [21]:
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
time.sleep(2)

In [24]:
# 6을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

In [25]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [27]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [28]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]